In [ ]:
import pandas as pd
import torch
import csv
import os
import gzip

In [ ]:
import pandas as pd
import torch
torch.set_float32_matmul_precision('high') 
import csv
import os
import gzip

DATA_DIR = "."
COMPRESSED_FILE = "mteval-task1-test25.tsv.gz"
DECOMPRESSED_FILE = "mteval-task1-test25.tsv"

compressed_path = os.path.join(DATA_DIR, COMPRESSED_FILE)
decompressed_path = os.path.join(DATA_DIR, DECOMPRESSED_FILE)

print(f"Looking for compressed file at: {compressed_path}")
if not os.path.exists(compressed_path):
    print(f"ERROR: File {COMPRESSED_FILE} not found in current directory!")
    print(f"Current directory contents: {os.listdir(DATA_DIR)}")
    raise FileNotFoundError(f"Required file {COMPRESSED_FILE} not found")
if not os.path.exists(decompressed_path):
    print(f"Decompressing {COMPRESSED_FILE}...")
    try:
        with gzip.open(compressed_path, 'rt', encoding='utf-8') as gz_file:
            with open(decompressed_path, 'w', encoding='utf-8') as out_file:
                for line in gz_file:
                    out_file.write(line)
        print(f"Successfully decompressed to {DECOMPRESSED_FILE}")
    except Exception as e:
        print(f"Failed to decompress file: {e}")
        raise
else:
    print(f"Using existing decompressed file: {DECOMPRESSED_FILE}")

print("Checking decompressed file content before loading...")
try:
    with open(decompressed_path, 'r', encoding='utf-8') as f:
        for i in range(5):
            line = f.readline()
            if not line:
                break
            print(f"Line {i+1}: {line.strip()}")
except Exception as e:
    print(f"Failed to read decompressed file: {e}")
    raise

print("Loading data into Pandas DataFrame...")
try:
    df_test = pd.read_csv(
        decompressed_path,
        sep='\t',
        header=0,
        encoding='utf-8',
        quoting=csv.QUOTE_NONE
    )
    print(f"Successfully loaded and parsed {len(df_test)} segments.")
    print("Data Schema Verified:")
    df_test.info()
    print("\nFirst 5 rows of test data:")
    print(df_test.head().to_string())
except Exception as e:
    print(f"Failed to load data into DataFrame: {e}")
    raise

Looking for compressed file at: .\mteval-task1-test25.tsv.gz
Using existing decompressed file: mteval-task1-test25.tsv
Checking decompressed file content before loading...
Line 1: doc_id	segment_id	source_lang	target_lang	set_id	system_id	source_segment	hypothesis_segment	reference_segment	domain_name	method
Line 2: en-is_IS_#_literary_#_rink_rats_chapter1	1	en	is_IS	official	AMI	Rink Rats \n Chapter 1: First Day \n Kyle looked at his reflection in the mirror. He studied the lines etched on his face, each one a reminder of the years gone by. A hint of gray threaded his hair now, and for a moment, he felt the weight of change settling in. \n He didn’t want to label it an identity crisis. Not yet at least. But as he splashed the cold sink water on his face, memories of his pro hockey career flooded his mind. The thrill of the game, the adrenaline of competition, the love of the fans. All forty years of his life had practically been dedicated to making it to the highest level. Now those d

C:\Users\97798\AppData\Local\Temp\ipykernel_10028\534853711.py:52: DtypeWarning: Columns (0,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(


Successfully loaded and parsed 742740 segments.
Data Schema Verified:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742740 entries, 0 to 742739
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   doc_id              742740 non-null  object
 1   segment_id          742740 non-null  int64 
 2   source_lang         742740 non-null  object
 3   target_lang         742740 non-null  object
 4   set_id              742740 non-null  object
 5   system_id           742740 non-null  object
 6   source_segment      742700 non-null  object
 7   hypothesis_segment  742698 non-null  object
 8   reference_segment   134242 non-null  object
 9   domain_name         733428 non-null  object
 10  method              137233 non-null  object
dtypes: int64(1), object(10)
memory usage: 62.3+ MB

First 5 rows of test data:
                                     doc_id  segment_id source_lang target_lang    set_id system_id   

In [ ]:
from comet import download_model, load_from_checkpoint

MODEL_NAME = "Unbabel/wmt22-comet-da"
BATCH_SIZE = 4
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Execution device: {device}")

print(f"Loading SOTA model: {MODEL_NAME}...")
model_path = download_model(MODEL_NAME)
model = load_from_checkpoint(model_path).to(device)
model.eval()

print("Formatting data for prediction...")
data_to_score = []
for _, row in df_test.iterrows():
    src = str(row['source_segment']).replace(' \\n ', '\n')
    hyp = str(row['hypothesis_segment']).replace(' \\n ', '\n')
    
    ref_val = row['reference_segment']
    if pd.isna(ref_val) or ref_val == "NaN":
        ref_content = ""
    else:
        ref_content = str(ref_val).replace(' \\n ', '\n')
        
    data_to_score.append({"src": src, "mt": hyp, "ref": ref_content})
print(f"Data formatting complete for {len(data_to_score)} entries.")

c:\Users\97798\Downloads\wmt_solution_2025\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Execution device: cuda
Loading SOTA model: Unbabel/wmt22-comet-da...


Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\97798\.cache\huggingface\hub\models--Unbabel--wmt22-comet-da\snapshots\2760a223ac957f30acfb18c8aa649b01cf1d75f2\checkpoints\model.ckpt`
Encoder model frozen.
c:\Users\97798\Downloads\wmt_solution_2025\venv\Lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


Formatting data for prediction...
Data formatting complete for 742740 entries.


In [ ]:
print("Starting prediction... This may take some time.")
with torch.no_grad():
    seg_scores, _ = model.predict(
        data_to_score,
        batch_size=BATCH_SIZE,
        gpus=1 if device == "cuda" else 0,
        progress_bar=True
    )
print(f"Prediction finished. Generated {len(seg_scores)} scores.")

Starting prediction... This may take some time.


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 185685/185685 [7:17:36<00:00,  7.07it/s]  


Prediction finished. Generated 6 scores.


In [ ]:
SUBMISSION_DIR = "."
SUBMISSION_FILE_PATH = os.path.join(SUBMISSION_DIR, "segments.tsv")

print("Creating the final submission DataFrame...")
submission_df = df_test[[
    'doc_id', 'segment_id', 'source_lang', 'target_lang',
    'set_id', 'system_id', 'domain_name', 'method'
]].copy()

submission_df['overall'] = seg_scores

print(f"Saving submission file to {SUBMISSION_FILE_PATH}...")
submission_df.to_csv(
    SUBMISSION_FILE_PATH,
    sep='\t',
    index=False,
    header=True,
    encoding='utf-8'
)
print("--- Submission file created successfully! ---")

print("\nVerifying submission file head:")
print(pd.read_csv(SUBMISSION_FILE_PATH, sep='\t').head().to_string())

Creating the final submission DataFrame...
Saving submission file to .\segments.tsv...
--- Submission file created successfully! ---

Verifying submission file head:
                                     doc_id  segment_id source_lang target_lang    set_id system_id domain_name method overall
0  en-is_IS_#_literary_#_rink_rats_chapter1           1          en       is_IS  official       AMI    literary    ESA  scores
1  en-is_IS_#_literary_#_rink_rats_chapter1           2          en       is_IS  official       AMI    literary    ESA  scores
2  en-is_IS_#_literary_#_rink_rats_chapter1           3          en       is_IS  official       AMI    literary    ESA  scores
3  en-is_IS_#_literary_#_rink_rats_chapter1           4          en       is_IS  official       AMI    literary    ESA  scores
4  en-is_IS_#_literary_#_rink_rats_chapter1           5          en       is_IS  official       AMI    literary    ESA  scores


C:\Users\97798\AppData\Local\Temp\ipykernel_10028\3793421129.py:29: DtypeWarning: Columns (0,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  print(pd.read_csv(SUBMISSION_FILE_PATH, sep='\t').head().to_string())
